In [1]:
import logging
from pathlib import Path
import polars as pl
import hvplot

from analyzepolar.loader import DeviceDataSource
from analyzepolar.logger import POLAR_ANALYZER_LOGGER
from analyzepolar.model import PolarDeviceData
from analyzer.common import PHASE_COLUMNS, Phase

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(threadName)s - %(levelname)s - %(name)s - %(message)s")
POLAR_ANALYZER_LOGGER.setLevel(logging.INFO)
_logger = POLAR_ANALYZER_LOGGER.getChild("notebook")
data_dir=Path("../data/")
assert data_dir.exists()

In [2]:

data = PolarDeviceData.load([DeviceDataSource(data_dir/"unten", "unten"), DeviceDataSource(data_dir/"oben", "oben")])

2024-08-25 15:44:36,360 - MainThread - INFO - polar analyzer.loader - Reading 21 files for device unten...
2024-08-25 15:44:38,229 - MainThread - INFO - polar analyzer.loader - Reading 20 files for device oben...
2024-08-25 15:44:40,065 - MainThread - INFO - polar analyzer.loader - Found 299418 rows for 2 devices...


In [3]:
_logger.info("Collecting data...")
df = data.phase_data.collect()
_logger.info(f"Found {len(df)} rows")
df.head(3)

2024-08-25 15:44:40,071 - MainThread - INFO - polar analyzer.notebook - Collecting data...
2024-08-25 15:44:46,400 - MainThread - INFO - polar analyzer.notebook - Found 898254 rows


timestamp,device,file,phase,total_act_energy,fund_act_energy,total_act_ret_energy,fund_act_ret_energy,lag_react_energy,lead_react_energy,max_act_power,min_act_power,max_aprt_power,min_aprt_power,max_voltage,min_voltage,avg_voltage,max_current,min_current,avg_current
"datetime[μs, UTC]",str,str,enum,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-05-13 10:36:00 UTC,"""unten""","""../data/unten/2024-05-25T17:09…","""a""",0.0352,0.0311,0.0,0.0,0.0,0.1286,7.4,6.6,42.4,33.2,233.812,233.335,233.532,0.181,0.142,0.146
2024-05-13 10:37:00 UTC,"""unten""","""../data/unten/2024-05-25T17:09…","""a""",0.1136,0.1142,0.0,0.0,0.0,0.4885,7.3,6.4,34.0,33.2,233.669,232.788,233.323,0.145,0.142,0.144
2024-05-13 10:38:00 UTC,"""unten""","""../data/unten/2024-05-25T17:09…","""a""",0.1136,0.1142,0.0,0.0,0.0,0.4862,7.5,6.5,33.8,33.0,233.379,231.913,232.791,0.145,0.142,0.143


In [10]:
data.total_energy_by_day_of_week().collect()

day_of_week,device,phase,total_act_energy
enum,str,enum,f64
"""Monday""","""unten""","""b""",649.470093
"""Monday""","""unten""","""c""",1378.461473
"""Monday""","""unten""","""a""",724.044213
"""Monday""","""oben""","""c""",108.759033
"""Monday""","""oben""","""a""",251.17648
…,…,…,…
"""Sunday""","""oben""","""c""",96.584273
"""Sunday""","""oben""","""a""",250.483693
"""Sunday""","""oben""","""b""",305.282473


In [16]:
data.total_energy_by_day_of_week().collect().plot.bar(x="day_of_week",y="total_act_energy",
                                                      by=["device", "phase"],
                                                      hover=True, xlabel="Day of week", ylabel="Total energy" )

:Bars   [day_of_week,device,phase]   (total_act_energy)

In [6]:
#data.plot_all("total_act_energy")
#data.plot("total_act_energy", Phase.B, "unten")
data.plot_total_energy("1d","unten",Phase.A)+data.plot_total_energy("1d","unten",Phase.B)+data.plot_total_energy("1d","unten",Phase.C)

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


:Layout
   .DynamicMap.I   :DynamicMap   []
      :Bars   [timestamp]   (total_act_energy)
   .DynamicMap.II  :DynamicMap   []
      :Bars   [timestamp]   (total_act_energy)
   .DynamicMap.III :DynamicMap   []
      :Bars   [timestamp]   (total_act_energy)

In [7]:
data.plot_total_energy("1d")

:DynamicMap   []
   :Bars   [timestamp]   (total_act_energy)